# Gateway MCP Testing Notebook

This notebook demonstrates how to:
1. Create a Cognito client with secret
2. Authenticate using Cognito
3. List available tools from the Gateway
4. Call the getWeather tool

In [19]:
import requests
import json
import boto3
from botocore.exceptions import ClientError

## Helper Functions

In [20]:
def get_stack_outputs(cf_client, stack_name):
    response = cf_client.describe_stacks(StackName=stack_name)
    outputs = {}
    for output in response['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

def get_or_create_m2m_client(cognito, user_pool_id, client_name):
    # Check if client already exists
    response = cognito.list_user_pool_clients(UserPoolId=user_pool_id, MaxResults=60)
    for client in response["UserPoolClients"]:
        if client["ClientName"] == client_name:
            describe = cognito.describe_user_pool_client(UserPoolId=user_pool_id, ClientId=client["ClientId"])
            return client["ClientId"], describe["UserPoolClient"]["ClientSecret"]
    
    print('Creating new m2m client')
    
    created = cognito.create_user_pool_client(
        UserPoolId=user_pool_id,
        ClientName=client_name,
        GenerateSecret=True
    )
    return created["UserPoolClient"]["ClientId"], created["UserPoolClient"]["ClientSecret"]

def get_token(user_pool_id, client_id, client_secret, scope_string, region):
    user_pool_id_without_underscore = user_pool_id.replace('_', '')
    token_url = f"https://{user_pool_id_without_underscore}.auth.{region}.amazoncognito.com/oauth2/token"
    
    response = requests.post(
        token_url,
        data=f"grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}&scope={scope_string}",
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching token: {response.status_code} - {response.text}")
        return None

def fetch_access_token(client_id, client_secret, user_pool_id):
    token_url = f"https://cognito-idp.us-east-1.amazonaws.com/{user_pool_id}"
    
    response = requests.post(
        token_url,
        data=f"grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}",
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print(f"Error fetching token: {response.status_code} - {response.text}")
        return None

def list_tools(gateway_url, access_token):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    payload = {
        "jsonrpc": "2.0",
        "id": "list-tools-request",
        "method": "tools/list"
    }
    
    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()

def call_tool(gateway_url, access_token, tool_name, arguments):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    payload = {
        "jsonrpc": "2.0",
        "id": "call-tool-request",
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": arguments
        }
    }
    
    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()

## Step 1: Initialize AWS Clients and Get Stack Info

In [21]:
# Initialize AWS clients
cf_client = boto3.client('cloudformation', region_name='us-east-1')
cognito_client = boto3.client('cognito-idp', region_name='us-east-1')

# Get stack outputs
stack_outputs = get_stack_outputs(cf_client, 'agentcore-policy-dev')
print("Stack Outputs:")
for key, value in stack_outputs.items():
    print(f"{key}: {value}")

Stack Outputs:
GatewayArn: arn:aws:bedrock-agentcore:us-east-1:249522321342:gateway/test-gateway-fu4ea8iahc
GatewayUrl: https://test-gateway-fu4ea8iahc.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
EngineId: test_policy_engine-iy790lwnpo
EngineArn: arn:aws:bedrock-agentcore:us-east-1:249522321342:policy-engine/test_policy_engine-iy790lwnpo


In [22]:
# Extract gateway URL and find User Pool
gateway_url = stack_outputs['GatewayUrl']

# Find User Pool ID
user_pools = cognito_client.list_user_pools(MaxResults=50)
user_pool_id = None
for pool in user_pools['UserPools']:
    if 'test-gateway' in pool['Name']:
        user_pool_id = pool['Id']
        break

print(f"User Pool ID: {user_pool_id}")
print(f"Gateway URL: {gateway_url}")

User Pool ID: us-east-1_bbAilznJm
Gateway URL: https://test-gateway-fu4ea8iahc.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp


## Step 2: Create/Get Cognito Client and Authenticate

In [ ]:
# Create or get existing client
CLIENT_NAME = "mcp-test-client"
REGION = "us-east-1"
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

client_id, client_secret = get_or_create_m2m_client(cognito_client, user_pool_id, CLIENT_NAME)
print(f"Client ID: {client_id}")
print(f"Client Secret: {client_secret[:10]}...")

# Get access token using get_token function
token_response = get_token(user_pool_id, client_id, client_secret, scopeString, REGION)
if token_response:
    token = token_response["access_token"]
    print("Token response:", token)
    access_token = token
else:
    print("Failed to obtain access token")
    access_token = None

Client ID: 7t0v5pmq1n4qmsm6sdnf95ns1b
Client Secret: otm6ncam54...


ConnectionError: HTTPSConnectionPool(host='us-east-1bbailznjm.auth.us-east-1.amazoncognito.com', port=443): Max retries exceeded with url: /oauth2/token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x121505880>: Failed to resolve 'us-east-1bbailznjm.auth.us-east-1.amazoncognito.com' ([Errno 8] nodename nor servname provided, or not known)"))

## Step 3: Test Gateway MCP Endpoints

In [ ]:
# List available tools
if access_token:
    tools = list_tools(gateway_url, access_token)
    print("Available tools:")
    print(json.dumps(tools, indent=2))

In [ ]:
# Test getWeather for different cities
cities = ["New York", "Tokyo", "Paris"]

if access_token:
    for city in cities:
        print(f"\nGetting weather for {city}:")
        result = call_tool(gateway_url, access_token, "getWeather", {"city": city})
        print(json.dumps(result, indent=2))

## Cleanup (Optional)

In [ ]:
# Optionally delete the test client
def cleanup_client(user_pool_id, client_id):
    try:
        cognito_client.delete_user_pool_client(
            UserPoolId=user_pool_id,
            ClientId=client_id
        )
        print(f"Deleted client {client_id}")
    except ClientError as e:
        print(f"Error deleting client: {e}")

# Uncomment to cleanup
# cleanup_client(user_pool_id, client_id)